Step 1 — Import Libraries

In [2]:
import pandas as pd
import numpy as np
import torch
from transformers import DistilBertTokenizerFast, DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


Step 2 — Load Sentiment140 Dataset

In [3]:


cols = ["sentiment", "id", "date", "query", "user", "text"]
df = pd.read_csv("training.1600000.processed.noemoticon.csv", encoding="latin-1", names=cols)

df = df[["sentiment", "text"]]
df["sentiment"] = df["sentiment"].map({0:0, 4:1})

df.head()


,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."


Step 3 — Use a smaller subset (so it trains fast)

In [4]:
df = df.sample(80000, random_state=42)

train_texts, test_texts, train_labels, test_labels = train_test_split(
    df["text"].tolist(),
    df["sentiment"].tolist(),
    test_size=0.2,
    random_state=42
)

Step 4 — Tokenize

In [5]:
tokenizer = DistilBertTokenizerFast.from_pretrained("distilbert-base-uncased")

train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=64)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=64)


Step 5 — Create PyTorch Dataset

In [6]:
class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
test_dataset = TweetDataset(test_encodings, test_labels)


Step 6 — Load DistilBERT

In [7]:
model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=2
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step 7 — Metrics

In [8]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average='binary')
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}


Step 8 — Training Setup

In [9]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=2,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    logging_dir="./logs",
    logging_steps=500,
    save_steps=1000,
    eval_steps=1000,
    do_train=True,
    do_eval=True
)


Step 9 — Train

In [10]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


c:\Users\moham\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,0.489800
1000,0.439900
1500,0.427500
2000,0.415000
2500,0.402900
3000,0.399900
3500,0.393800
4000,0.390500
4500,0.292100
5000,0.267500


c:\Users\moham\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=8000, training_loss=0.34329015445709227, metrics={'train_runtime': 16385.5946, 'train_samples_per_second': 7.812, 'train_steps_per_second': 0.488, 'total_flos': 2119478378496000.0, 'train_loss': 0.34329015445709227, 'epoch': 2.0})

Step 10 — Evaluate

In [11]:
trainer.evaluate()


c:\Users\moham\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\utils\data\dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


{'eval_loss': 0.4194168746471405,
 'eval_accuracy': 0.8391875,
 'eval_f1': 0.8371209723365196,
 'eval_precision': 0.8474750064086132,
 'eval_recall': 0.827016885553471,
 'eval_runtime': 627.9287,
 'eval_samples_per_second': 25.481,
 'eval_steps_per_second': 1.593,
 'epoch': 2.0}

Step 11 — Test with your own text

In [12]:
def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = model(**inputs)
    pred = torch.argmax(outputs.logits).item()
    return "Positive 😊" if pred == 1 else "Negative 😡"

predict_sentiment("This is the worst service ever.")


'Negative 😡'

In [ ]:
# =========================
# SAVE / LOAD / TEST (ADD THIS AT THE END OF YOUR NOTEBOOK)
# =========================

import os
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Folder to save your fine-tuned model
SAVE_DIR = "sentiment_distilbert_finetuned"
os.makedirs(SAVE_DIR, exist_ok=True)

# ---------
# 1) SAVE
# ---------
# (trainer and tokenizer must already exist from your training cells)
trainer.save_model(SAVE_DIR)               # saves model + config
tokenizer.save_pretrained(SAVE_DIR)        # saves tokenizer files
print(f"✅ Saved model + tokenizer to: {SAVE_DIR}")

# ---------
# 2) LOAD
# ---------
loaded_tokenizer = AutoTokenizer.from_pretrained(SAVE_DIR)
loaded_model = AutoModelForSequenceClassification.from_pretrained(SAVE_DIR)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
loaded_model.to(device)
loaded_model.eval()
print(f"✅ Loaded model from: {SAVE_DIR} (device: {device})")

# ---------
# 3) TEST with your own texts
# ---------
# label mapping (your training: 0 = negative, 1 = positive)
id2label = {0: "NEGATIVE", 1: "POSITIVE"}

@torch.no_grad()
def predict_sentiment_texts(texts, max_length=128):
    """
    texts: list[str]
    returns: list of dicts: [{"text":..., "label":..., "score":...}, ...]
    """
    enc = loaded_tokenizer(
        texts,
        truncation=True,
        padding=True,
        max_length=max_length,
        return_tensors="pt"
    ).to(device)

    outputs = loaded_model(**enc)
    probs = torch.softmax(outputs.logits, dim=-1)  # shape: [B, 2]
    pred_ids = torch.argmax(probs, dim=-1).tolist()

    results = []
    for t, pid in zip(texts, pred_ids):
        score = float(probs[results.__len__(), pid].item())
        results.append({"text": t, "label": id2label[pid], "score": score})
    return results

# ---- Example usage (replace these with your own texts)
my_texts = [
    "This product is amazing, I love it!",
    "Worst experience ever. Very disappointed.",
    "It's okay, not bad but not great."
]

preds = predict_sentiment_texts(my_texts)
for p in preds:
    print(f"{p['label']:>8}  score={p['score']:.4f}  |  {p['text']}")

# ---- Optional: interactive testing (type your sentence)
while True:
    s = input("\nType a sentence to test (or type 'q' to quit): ").strip()
    if s.lower() in ("q", "quit", "exit"):
        break
    out = predict_sentiment_texts([s])[0]
    print(f"➡️  {out['label']} (score={out['score']:.4f})")


✅ Saved model + tokenizer to: sentiment_distilbert_finetuned
✅ Loaded model from: sentiment_distilbert_finetuned (device: cpu)
POSITIVE  score=0.9972  |  This product is amazing, I love it!
NEGATIVE  score=0.9971  |  Worst experience ever. Very disappointed.
POSITIVE  score=0.9630  |  It's okay, not bad but not great.
➡️  NEGATIVE (score=0.9935)
➡️  POSITIVE (score=0.9947)
➡️  NEGATIVE (score=0.9968)
